<a href="https://colab.research.google.com/github/garjita63/de-zoomcamp-2024-homework-workshop-data-ingestion/blob/main/homework_data_talks_club_data_extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)
sum_gen = 0

for sqrt_value in generator:
    print(sqrt_value)
    sum_gen =  sum_gen + sqrt_value

print("sum of the outputs of the generator for limit = 5 ==>",sum_gen)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
sum of the outputs of the generator for limit = 5 ==> 8.382332347441762


In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

print(" 13th number yielded by the generator ==>",sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989
 13th number yielded by the generator ==> 3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [23]:
from typing import AsyncGenerator
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

# Calculate the sum of all ages of people.
import pandas as pd
labels = ['Age']
sum_ages = 0
for person in people_1():
  df = pd.DataFrame(person, index=labels)
  sum_ages = sum_ages + int(df['Age'])

for person in people_2():
  df = pd.DataFrame(person, index=labels)
  sum_ages = sum_ages + int(df['Age'])

print("Sum of Ages ==>", sum_ages)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
Sum of Ages ==> 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [24]:
#Install the dependencies
#%%capture
#!pip install dlt[duckdb]

In [89]:
import dlt

pipeline = dlt.pipeline(pipeline_name='person_data',
						destination='duckdb',
						dataset_name='people_dataset')

from typing import AsyncGenerator

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for data in people_1():
  print(data)
  info = pipeline.run(data.values(), table_name="person", write_disposition="replace")

print(info)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
Pipeline people_data load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_dataset
The duckdb destination used duckdb:////content/people_data.duckdb location to store data
Load package 1707311475.4290483 is LOADED and contains no failed jobs


In [104]:
%%time
con.execute("SHOW ALL TABLES").df()

CPU times: user 22.6 ms, sys: 760 µs, total: 23.4 ms
Wall time: 17.4 ms


,database,schema,name,column_names,column_types,temporary


In [94]:
%%time
con.execute("select * from people_dataset.ketut").df()

CatalogException: Catalog Error: Table with name ketut does not exist!
Did you mean "temp.pg_catalog.pg_enum"?
LINE 1: select * from people_dataset.ketut
                      ^

In [87]:
# to do: homework :)

import dlt

pipeline = dlt.pipeline(pipeline_name='people_data',
						destination='duckdb',
						dataset_name='people_dataset')

from typing import AsyncGenerator

data = {"ID": [],
        "Name": [],
        "Age": [],
        "City": [],
        "Occupation": []
        }

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    data["ID"].append(person["ID"])
    data["Name"].append(person["Name"])
    data["Age"].append(person["Age"])
    data["City"].append(person["City"])

print(data)

import json
# Convert and write JSON object to file
with open("people.json", "w") as outfile:
  json.dump(data, outfile)

f = open('people.json')

data = json.load(f)

info = pipeline.run(data.values(), table_name="person", write_disposition="replace")

print(info)


{'ID': [1, 2, 3, 4, 5], 'Name': ['Person_1', 'Person_2', 'Person_3', 'Person_4', 'Person_5'], 'Age': [26, 27, 28, 29, 30], 'City': ['City_A', 'City_A', 'City_A', 'City_A', 'City_A'], 'Occupation': []}
Pipeline people_data load step completed in 0.34 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_dataset
The duckdb destination used duckdb:////content/people_data.duckdb location to store data
Load package 1707308237.782451 is LOADED and contains no failed jobs


In [78]:
f = open('people.json')

In [79]:
data = json.load(f)

In [81]:
print(data)

{'ID': [1, 2, 3, 4, 5], 'Name': ['Person_1', 'Person_2', 'Person_3', 'Person_4', 'Person_5'], 'Age': [26, 27, 28, 29, 30], 'City': ['City_A', 'City_A', 'City_A', 'City_A', 'City_A'], 'Occupation': []}


In [83]:
import dlt
import duckdb

pipeline = dlt.pipeline(pipeline_name='people_data',
						destination='duckdb',
						dataset_name='people_dataset')


In [85]:
info = pipeline.run(data.values(), table_name="people4", write_disposition="replace")

In [86]:
print(info)

Pipeline people_data load step completed in 0.30 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_dataset
The duckdb destination used duckdb:////content/people_data.duckdb location to store data
Load package 1707308022.2401197 is LOADED and contains no failed jobs


In [73]:
# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
import dlt
import duckdb

pipeline = dlt.pipeline(pipeline_name='people_data',
						destination='duckdb',
						dataset_name='people_dataset')

# run the pipeline with default settings, and capture the outcome
from typing import AsyncGenerator

data = {"ID": [],
        "Name": [],
        "Age": [],
        "City": [],
        "Occupation": []
        }

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
  print(person)

info = pipeline.run(person, table_name="people4", write_disposition="replace")

# show the outcome
print(info)


Pipeline people_data load step completed in 0.43 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_dataset
The duckdb destination used duckdb:////content/people_data.duckdb location to store data
Load package 1707305693.1921659 is LOADED and contains no failed jobs


In [76]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}
        info = pipeline.run(data, table_name="people4", write_disposition="replace")

    print(info)



IndentationError: unindent does not match any outer indentation level (<tokenize>, line 6)

In [68]:
import duckdb
con = duckdb.connect()

In [90]:
%%time
con.execute("SHOW ALL TABLES").df()

CPU times: user 22.1 ms, sys: 616 µs, total: 22.7 ms
Wall time: 19.5 ms


,database,schema,name,column_names,column_types,temporary


In [ ]:
con.e

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX